In [3]:
import os
# List the contents of the "data" directory
train_path = "cardata/images/training_images/"
data_directory_contents = os.listdir(train_path)
data_directory_contents[:5]

['vid_4_18340.jpg',
 'vid_4_2460.jpg',
 'vid_4_10720.jpg',
 'vid_4_14220.jpg',
 'vid_4_6160.jpg']

In [4]:
import os
test_path = "cardata/images/testing_images/"
data_directory_contents = os.listdir(test_path)
data_directory_contents[:5]

['vid_5_30040.jpg',
 'vid_5_27500.jpg',
 'vid_5_28380.jpg',
 'vid_5_26420.jpg',
 'vid_5_31160.jpg']

In [5]:
import pandas as pd

# Load the bounding boxes CSV file
bounding_boxes_csv_path = "cardata/labels/train_solution_bounding_boxes.csv"
bounding_boxes_df = pd.read_csv(bounding_boxes_csv_path)

# Display the first few rows of the CSV file
bounding_boxes_df.head()

,image,xmin,ymin,xmax,ymax
0,vid_4_1000.jpg,281.259045,187.035071,327.727931,223.225547
1,vid_4_10000.jpg,15.163531,187.035071,120.329957,236.430180
2,vid_4_10040.jpg,239.192475,176.764801,361.968162,236.430180
3,vid_4_10020.jpg,496.483358,172.363256,630.020260,231.539575
4,vid_4_10060.jpg,16.630970,186.546010,132.558611,238.386422


# To YOLO Format

In [7]:
from PIL import Image
import os

def convert_to_yolo_format(row, img_folder_path):
    """
    Convert the bounding box format to YOLO format.
    YOLO format: <class> <x_center> <y_center> <width> <height>
    All the values are normalized between 0 and 1 with respect to the image's width and height.
    """
    # Open the image and get its size
    img_path = os.path.join(img_folder_path, row['image'])
    with Image.open(img_path) as img:
        img_width, img_height = img.size

    # Calculate bounding box parameters
    x_center = (row['xmin'] + row['xmax']) / 2.0
    y_center = (row['ymin'] + row['ymax']) / 2.0
    width = row['xmax'] - row['xmin']
    height = row['ymax'] - row['ymin']

    # Normalize the values
    x_center /= img_width
    y_center /= img_height
    width /= img_width
    height /= img_height

    # Class index (assuming a single class dataset)
    class_index = 0

    return f"{class_index} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}"

# Test the function on the first row of the dataframe
yolo_format_example = convert_to_yolo_format(bounding_boxes_df.iloc[0], train_path)
"yolo formate one sample : " , yolo_format_example

('yolo formate one sample : ', '0 0.450434 0.539817 0.068741 0.095238')

In [9]:
# Create YOLO formatted annotations and save them as .txt files
output_folder = "cardata/labels/train/"
# Ensure the output directory exists
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

for _, row in bounding_boxes_df.iterrows():
    yolo_data = convert_to_yolo_format(row,train_path)
    
    # Generate output file path
    output_file_name = os.path.splitext(row['image'])[0] + '.txt'
    output_path = os.path.join(output_folder, output_file_name)
    
    # Write to the file
    with open(output_path, 'a') as file:
        file.write(yolo_data + '\n')

# Check the first few files in the output directory
yolo_annotations_files = os.listdir(output_folder)[:5]
"examples : "  , yolo_annotations_files

('examples : ',
 ['vid_4_740.txt',
  'vid_4_6260.txt',
  'vid_4_13840.txt',
  'vid_4_15000.txt',
  'vid_4_21400.txt'])